#RxId : CSV Data Cleaning

Data Source : https://pillbox.nlm.nih.gov/developers.html
https://dev.socrata.com/foundry/datadiscovery.nlm.nih.gov/crzr-uvwg


Issue :  Two CSV files were downloaded from the above site.  

    Pillbox.NO.ID.csv  has useful meds data but no image_id  field to link it to a filename in the image library.

    Pillbox.NO.IMAGE ID.csv  lacks useful meds data but HAS image_id  field to link it to a filename in the image library.

Solution:   Clean CSV files individually and merge into single CSV that will be used to load an AWS RDS database


### Load CSVs into dataframes

In [ ]:
import pandas as pd
pd.options.display.max_columns = None

In [100]:
url1="https://raw.githubusercontent.com/labs12-rxid/DS/master/CSV/Pillbox.NO.ID.csv"
df1=pd.read_csv(url1)

url2="https://raw.githubusercontent.com/labs12-rxid/DS/master/CSV/Pillbox.IMAGE%20ID.csv"
df2=pd.read_csv(url2)


print(df1.shape)
df2.shape

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (62,63,64,65,66,69,70,71) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(62898, 72)


(62898, 24)

In [101]:
df1.columns

Index(['ID', 'Enabled?', 'created at', 'updated at', 'spp', 'setid', 'splsize',
       'pillbox_size', 'splshape', 'splshape_text', 'pillbox_shape_text',
       'splscore', 'pillbox_score', 'splimprint', 'pillbox_imprint',
       'splcolor', 'splcolor_text', 'pillbox_color_text', 'spl_strength',
       'spl_ingredients', 'spl_inactive_ing', 'source', 'rxtty', 'rxstring',
       'rxcui', 'product_code', 'part_num', 'part_medicine_name', 'ndc9',
       'ndc_labeler_code', 'ndc_product_code', 'medicine_name',
       'marketing_act_code', 'effective_time', 'file_name',
       'equal_product_code', 'dosage_form', 'document_type',
       'dea_schedule_code', 'dea_schedule_name', 'author_type', 'author',
       'approval_code', 'image_source', 'splimage', 'has_image', 'epc_match',
       'version_number', 'laberer_code', 'application_number', 'updated',
       'stale', 'new', 'Unnamed: 53', 'Unnamed: 54', 'Unnamed: 55',
       'Unnamed: 56', 'Unnamed: 57', 'Unnamed: 58', 'Unnamed: 59',
      

###  Rename df2.id to df2.ID  to match df1

In [102]:
df2.rename(columns={'id':'ID'}, inplace=True)
df2.columns

Index(['ID', 'spl_id', 'SETID', 'spp', 'NDC9', 'PRODUCT_CODE',
       'EQUAL_PRODUCT_CODE', 'author', 'SPLIMPRINT', 'SPLCOLOR', 'SPLSHAPE',
       'SPLSIZE', 'SPLSCORE', 'DEA_SCHEDULE_CODE', 'INGREDIENTS',
       'SPL_INACTIVE_ING', 'RXCUI', 'RXTTY', 'RXSTRING', 'image_id',
       'IMAGE_SOURCE', 'HAS_IMAGE', 'FROM_SIS', 'NO_RXCUI'],
      dtype='object')

### Drop Useless/Duplicated Columns from df1 & df2

In [103]:
drop_col_1=['created at', 'updated at', 'ndc9', 'author',
            'rxstring', 'has_image',
       'Unnamed: 53', 'Unnamed: 54', 'Unnamed: 55',
       'Unnamed: 56', 'Unnamed: 57', 'Unnamed: 58', 'Unnamed: 59',
       'Unnamed: 60', 'Unnamed: 61', 'Unnamed: 62', 'Unnamed: 63',
       'Unnamed: 64', 'Unnamed: 65', 'Unnamed: 66', 'Unnamed: 67',
       'Unnamed: 68', 'Unnamed: 69', 'Unnamed: 70', 'Unnamed: 71']
df1.drop(columns=drop_col_1, inplace=True)
df1.columns

Index(['ID', 'Enabled?', 'spp', 'setid', 'splsize', 'pillbox_size', 'splshape',
       'splshape_text', 'pillbox_shape_text', 'splscore', 'pillbox_score',
       'splimprint', 'pillbox_imprint', 'splcolor', 'splcolor_text',
       'pillbox_color_text', 'spl_strength', 'spl_ingredients',
       'spl_inactive_ing', 'source', 'rxtty', 'rxcui', 'product_code',
       'part_num', 'part_medicine_name', 'ndc_labeler_code',
       'ndc_product_code', 'medicine_name', 'marketing_act_code',
       'effective_time', 'file_name', 'equal_product_code', 'dosage_form',
       'document_type', 'dea_schedule_code', 'dea_schedule_name',
       'author_type', 'approval_code', 'image_source', 'splimage', 'epc_match',
       'version_number', 'laberer_code', 'application_number', 'updated',
       'stale', 'new'],
      dtype='object')

In [104]:
drop_col_2=['SETID', 'spp', 'INGREDIENTS','SPL_INACTIVE_ING','SPLSIZE', 'SPLSCORE',
           'SPLIMPRINT', 'SPLCOLOR',	'SPLSHAPE', 'RXCUI',	'RXTTY', 'IMAGE_SOURCE']
df2.drop(columns=drop_col_2, inplace=True)
df2.columns

Index(['ID', 'spl_id', 'NDC9', 'PRODUCT_CODE', 'EQUAL_PRODUCT_CODE', 'author',
       'DEA_SCHEDULE_CODE', 'RXSTRING', 'image_id', 'HAS_IMAGE', 'FROM_SIS',
       'NO_RXCUI'],
      dtype='object')

### compare same  ID acrross Frames

In [105]:
df1.query('ID==3143')

,ID,Enabled?,spp,setid,splsize,pillbox_size,splshape,splshape_text,pillbox_shape_text,splscore,pillbox_score,splimprint,pillbox_imprint,splcolor,splcolor_text,pillbox_color_text,spl_strength,spl_ingredients,spl_inactive_ing,source,rxtty,rxcui,product_code,part_num,part_medicine_name,ndc_labeler_code,ndc_product_code,medicine_name,marketing_act_code,effective_time,file_name,equal_product_code,dosage_form,document_type,dea_schedule_code,dea_schedule_name,author_type,approval_code,image_source,splimage,epc_match,version_number,laberer_code,application_number,updated,stale,new
25440,3143,True,e4682a6e-9624-48b0-978f-95da00cfb78f-0093-7372-0,e4682a6e-9624-48b0-978f-95da00cfb78f,19.0,NaN,C48336,CAPSULE,NaN,1.0,NaN,TEVA;7372,NaN,C48328;C48325,PINK;WHITE,NaN,AMLODIPINE BESYLATE 5 mg;BENAZEPRIL HYDROCHLOR...,AMLODIPINE BESYLATE[AMLODIPINE];BENAZEPRIL HYD...,FERROSOFERRIC OXIDE;ANHYDROUS DIBASIC CALCIUM ...,CORN;CROSPOVIDONE (15 MPA.S AT 5%);GELATIN,UNSPECIFIED;LACTOSE MONOHYDRATE;MAGNESIUM STE...,SCD,Amlodipine 5 MG / Benazepril hydrochloride 20 ...,898356,0093-7372,NaN,937372,93,7372,Amlodipine and Benazepril Hydrochloride,active,20170823,a804a97e-d1e0-411b-b075-557ac09684c0.xml,NaN,C25158,34391-3,NaN,LABELER,Teva Pharmaceuticals USA,Inc.,SPL,93737201,TRUE,NaN,16,NaN,NaN


In [106]:
df2.query('ID==3143')

,ID,spl_id,NDC9,PRODUCT_CODE,EQUAL_PRODUCT_CODE,author,DEA_SCHEDULE_CODE,RXSTRING,image_id,HAS_IMAGE,FROM_SIS,NO_RXCUI
62894,3143,NaN,000937372,0093-7372,NaN,"Teva Pharmaceuticals USA, Inc.",NaN,Amlodipine 5 MG / Benazepril hydrochloride 20 ...,00093737201,1,NaN,NaN


###  Combine dataframes

In [ ]:
df_comb=pd.merge(df1,df2,how='left', on=['ID'])

In [111]:
df_comb.query('ID==3143')

,ID,Enabled?,spp,setid,splsize,pillbox_size,splshape,splshape_text,pillbox_shape_text,splscore,pillbox_score,splimprint,pillbox_imprint,splcolor,splcolor_text,pillbox_color_text,spl_strength,spl_ingredients,spl_inactive_ing,source,rxtty,rxcui,product_code,part_num,part_medicine_name,ndc_labeler_code,ndc_product_code,medicine_name,marketing_act_code,effective_time,file_name,equal_product_code,dosage_form,document_type,dea_schedule_code,dea_schedule_name,author_type,approval_code,image_source,splimage,epc_match,version_number,laberer_code,application_number,updated,stale,new,spl_id,NDC9,PRODUCT_CODE,EQUAL_PRODUCT_CODE,author,DEA_SCHEDULE_CODE,RXSTRING,image_id,HAS_IMAGE,FROM_SIS,NO_RXCUI
25440,3143,True,e4682a6e-9624-48b0-978f-95da00cfb78f-0093-7372-0,e4682a6e-9624-48b0-978f-95da00cfb78f,19.0,NaN,C48336,CAPSULE,NaN,1.0,NaN,TEVA;7372,NaN,C48328;C48325,PINK;WHITE,NaN,AMLODIPINE BESYLATE 5 mg;BENAZEPRIL HYDROCHLOR...,AMLODIPINE BESYLATE[AMLODIPINE];BENAZEPRIL HYD...,FERROSOFERRIC OXIDE;ANHYDROUS DIBASIC CALCIUM ...,CORN;CROSPOVIDONE (15 MPA.S AT 5%);GELATIN,UNSPECIFIED;LACTOSE MONOHYDRATE;MAGNESIUM STE...,SCD,Amlodipine 5 MG / Benazepril hydrochloride 20 ...,898356,0093-7372,NaN,937372,93,7372,Amlodipine and Benazepril Hydrochloride,active,20170823,a804a97e-d1e0-411b-b075-557ac09684c0.xml,NaN,C25158,34391-3,NaN,LABELER,Teva Pharmaceuticals USA,Inc.,SPL,93737201,TRUE,NaN,16,NaN,NaN,NaN,000937372,0093-7372,NaN,"Teva Pharmaceuticals USA, Inc.",NaN,Amlodipine 5 MG / Benazepril hydrochloride 20 ...,00093737201,1,NaN,NaN


### Write out combined CSV

In [ ]:
df_comb.to_csv('Pills.Final.csv', index=False ) #header=['id','status_group'])

In [2]:
    # ___ load the CSV into a df ____
    import pandas as pd
    csv_url = "Pills.Final.csv"
    df = pd.read_csv(csv_url)

C:\Users\EDC\Anaconda3\envs\environ1\lib\site-packages\IPython\core\interactiveshell.py:3044: DtypeWarning: Columns (45) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Find all the unique color values in the splcolor_text field

In [1]:
import pandas as pd
pd.options.display.max_columns = None

In [3]:
url_final="Pills.Final.csv"
df_color=pd.read_csv(url_final)

C:\Users\EDC\Anaconda3\envs\environ1\lib\site-packages\IPython\core\interactiveshell.py:3044: DtypeWarning: Columns (45) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
pd.options.display.max_rows = None

df_color['splcolor_text'].value_counts()

In [13]:
df_color['splshape_text'].unique()

array(['ROUND', 'OVAL', 'CAPSULE', 'DIAMOND', 'TRIANGLE',
       'PENTAGON (5 SIDED)', 'HEXAGON (6 SIDED)', 'RECTANGLE', 'BULLET',
       'FREEFORM', 'SQUARE', 'OCTAGON (8 SIDED)', 'TRAPEZOID',
       'DOUBLE CIRCLE', 'TEAR', 'SEMI-CIRCLE', 'CLOVER', nan],
      dtype=object)